# Assignment 1
## Question `1` (K-Nearest Neighbour)

| | |
|-|-|
| Course | Statistical Methods in AI |
| Release Date | `19.01.2023` |
| Due Date | `29.01.2023` |

### Instructions:
1.   Assignment must be implemented using python notebook only (Colab , VsCode , Jupyter etc.)
2.   You are allowed to use libraries for data preprocessing (numpy, pandas, nltk etc) and for algorithms as well (sklearn etc). You are not however allowed to directly use classifier models.
3.   The performance of the model will hold weightage but you will also be graded largely for data preprocessing steps , explanations , feature selection for vectors etc.
4.   Strict plagiarism checking will be done. An F will be awarded for plagiarism.

### The Dataset
The dataset is avaible in the zip file which is a collection of *11099 tweets*. The data will be in the form of a csv file. The ground truth is also given in the zip file which corresponds to whether a tweet was popular or not. Since the task involves selecting features yourself to vectorize a tweet , we suggest some data analysis of the columns you consider important.
<br><br>

### The Task
You have to build a classifier which can predict the popularity of the tweet, i.e , if the tweet was popular or not. You are required to use **KNN** algorithm to build the classifier and cannot use any inbuilt classifier. All columns are supposed to be analyzed , filtered and preprocessed to determine its importance as a feature in the vector for every tweet (Not every column will be useful).<br>
The Data contains the **raw text of the tweet**(in the text column) as well as other **meta data** like likes count , user followers count. Note that it might be useful to **create new columns** with useful information. For example, *number of hashtags* might be useful but is not directly present as a column.<br>
There are 3 main sub parts:
1. *Vectorize tweets using only meta data* - likes , user followers count , and other created data
2. *Vectorize tweets using only it's text*. This segment will require NLP techniques to clean the text and extract a vector using a BoW model. Here is a useful link for the same - [Tf-Idf](https://towardsdatascience.com/text-vectorization-term-frequency-inverse-document-frequency-tfidf-5a3f9604da6d). Since these vectors will be very large , we recommend reducing their dimensinality (~10 - 25). Hint: [Dimentionality Reduction](https://jonathan-hui.medium.com/machine-learning-singular-value-decomposition-svd-principal-component-analysis-pca-1d45e885e491). Please note that for this also you are allowed to use libraries.

3. *Combining the vectors from above two techinques to create one bigger vector*
<br>


Using KNN on these vectors build a classifier to predict the popularity of the tweet and report accuracies on each of the three methods as well as analysis. You can use sklearn's Nearest Neighbors and need not write KNN from scratch. (However you cannot use the classifier directly). You are expected to try the classifier for different number of neighbors and identify the optimal K value.

## Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize
from collections import Counter
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

## Load and display the data

In [3]:
#your code here
df = pd.read_csv('/content/Tweets.csv')
df

,created_at,id,id_str,text,truncated,entities,metadata,source,is_quote_status,retweet_count,...,lang,user_name,user_screen_name,user_followers_count,user_friends_count,user_listed_count,user_created_at,user_favourites_count,user_verified,user_statuses_count
0,Tue Jul 31 13:34:34 +0000 2018,1.024290e+18,1.024290e+18,RT @EdwardTHardy: The 7th US Circuit Court of ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,113,...,en,Sherry Wahl,queenfancygirl,153,264,7,Thu Mar 18 19:16:31 +0000 2010,32984,False,31308
1,Tue Jul 31 13:34:14 +0000 2018,1.024290e+18,1.024290e+18,RT @VenomMovie: The world has enough superhero...,False,"{'hashtags': [{'text': 'Venom', 'indices': [64...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,5902,...,en,Kay Khairil ðŸŒ,ikaykhairil,780,382,12,Wed Mar 17 03:27:51 +0000 2010,6648,False,87272
2,Tue Jul 31 13:34:40 +0000 2018,1.024290e+18,1.024290e+18,RT @FutbolBible: Teachers vs Students match &a...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",False,3745,...,en,Charlie Hamilton,ch100897,255,246,1,Sun Mar 03 09:23:03 +0000 2013,5426,False,1731
3,Tue Jul 31 13:34:27 +0000 2018,1.024290e+18,1.024290e+18,RT @mashable: Someone from 'The Office' actual...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,10,...,en,Mike Santos,mikesantosx71,2419,2428,4,Thu May 25 14:37:29 +0000 2017,5993,False,2153
4,Tue Jul 31 13:34:28 +0000 2018,1.024290e+18,1.024290e+18,RT @_missj0hnson: Iâ€™m at Starbucks asking fo...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,25306,...,en,Soots,DaAverageDingus,314,722,7,Tue Mar 15 01:14:02 +0000 2011,6285,False,33503
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11094,Tue Jul 31 13:34:36 +0000 2018,1.024290e+18,1.024290e+18,RT @AnnieOakley2015: @AVoiceForAllGA Following...,False,"{'hashtags': [{'text': 'GA10', 'indices': [93,...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,5,...,en,TrumpNeck,Francis_3,802,1353,7,Thu May 28 12:02:54 +0000 2009,79149,False,6491
11095,Tue Jul 31 13:34:15 +0000 2018,1.024290e+18,1.024290e+18,RT @BillOReilly: What exactly is Donald Trump ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",False,8393,...,en,atm,allen_mcdowell,429,599,12,Wed Oct 10 11:37:34 +0000 2012,2029,False,5065
11096,Tue Jul 31 13:34:14 +0000 2018,1.024290e+18,1.024290e+18,"Paradoxically, your good intentions may stand ...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.twittascope.com"" rel=""nofo...",False,0,...,en,Jan15î„… .,yellow_naeeeee,135,94,0,Mon Sep 23 22:37:49 +0000 2013,90,False,4265
11097,Tue Jul 31 13:34:35 +0000 2018,1.024290e+18,1.024290e+18,RT @enblocibc: Less than 4 days to go for the ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,18,...,en,âœ¨MC Capital Venturesâœ¨,MCCapitalV,505,243,8,Thu Feb 01 23:58:16 +0000 2018,15829,False,13604


In [4]:
df_truth = pd.read_csv('/content/ground_truth.csv')
df_truth.columns = ['Ground Truth']
df_truth['Ground Truth'] = df_truth['Ground Truth'].astype('int')
df_truth

,Ground Truth
0,0
1,0
2,1
3,0
4,1
...,...
11094,0
11095,1
11096,0
11097,0


## Exploratory Data Analysis
*This is an ungraded section but is recommended to get a good grasp on the dataset*

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11099 entries, 0 to 11098
Data columns (total 21 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   created_at             11099 non-null  object 
 1   id                     11099 non-null  float64
 2   id_str                 11099 non-null  float64
 3   text                   11099 non-null  object 
 4   truncated              11099 non-null  bool   
 5   entities               11099 non-null  object 
 6   metadata               11099 non-null  object 
 7   source                 11099 non-null  object 
 8   is_quote_status        11099 non-null  bool   
 9   retweet_count          11099 non-null  int64  
 10  favorite_count         11099 non-null  int64  
 11  lang                   11099 non-null  object 
 12  user_name              11099 non-null  object 
 13  user_screen_name       11099 non-null  object 
 14  user_followers_count   11099 non-null  int64  
 15  us

In [6]:
df.describe()

,id,id_str,retweet_count,favorite_count,user_followers_count,user_friends_count,user_listed_count,user_favourites_count,user_statuses_count
count,1.109900e+04,1.109900e+04,11099.000000,11099.000000,1.109900e+04,11099.000000,11099.000000,11099.000000,1.109900e+04
mean,1.024290e+18,1.024290e+18,2777.956392,0.039463,6.009168e+03,1442.336337,47.017479,19413.978286,3.476254e+04
std,0.000000e+00,0.000000e+00,12180.169923,1.013902,2.013144e+05,7645.949991,254.953725,39144.906425,8.879138e+04
min,1.024290e+18,1.024290e+18,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,1.000000e+00
25%,1.024290e+18,1.024290e+18,0.000000,0.000000,1.310000e+02,194.000000,0.000000,1052.000000,2.543000e+03
50%,1.024290e+18,1.024290e+18,13.000000,0.000000,4.030000e+02,442.000000,3.000000,5538.000000,9.943000e+03
75%,1.024290e+18,1.024290e+18,428.500000,0.000000,1.249000e+03,1116.000000,19.000000,19576.500000,3.418700e+04
max,1.024290e+18,1.024290e+18,413719.000000,101.000000,2.021186e+07,510292.000000,12895.000000,635920.000000,2.848360e+06


In [7]:
# your code here
df.dropna(inplace = True) # none found
x = df.duplicated()
for i in x:
    if(int(i) == 1):
        print(i) # no duplicates found

### Adding new columns

In [8]:
df['entities'] = df['entities'].apply(lambda x: eval(x))
# print(df['entities'][0].keys(), df['entities'][0].values())
hashtags = list()
for i in df['entities']:
    hashtags.append(len(i['hashtags']))
    
df['hashtags'] = hashtags
df

,created_at,id,id_str,text,truncated,entities,metadata,source,is_quote_status,retweet_count,...,user_name,user_screen_name,user_followers_count,user_friends_count,user_listed_count,user_created_at,user_favourites_count,user_verified,user_statuses_count,hashtags
0,Tue Jul 31 13:34:34 +0000 2018,1.024290e+18,1.024290e+18,RT @EdwardTHardy: The 7th US Circuit Court of ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,113,...,Sherry Wahl,queenfancygirl,153,264,7,Thu Mar 18 19:16:31 +0000 2010,32984,False,31308,0
1,Tue Jul 31 13:34:14 +0000 2018,1.024290e+18,1.024290e+18,RT @VenomMovie: The world has enough superhero...,False,"{'hashtags': [{'text': 'Venom', 'indices': [64...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,5902,...,Kay Khairil ðŸŒ,ikaykhairil,780,382,12,Wed Mar 17 03:27:51 +0000 2010,6648,False,87272,1
2,Tue Jul 31 13:34:40 +0000 2018,1.024290e+18,1.024290e+18,RT @FutbolBible: Teachers vs Students match &a...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",False,3745,...,Charlie Hamilton,ch100897,255,246,1,Sun Mar 03 09:23:03 +0000 2013,5426,False,1731,0
3,Tue Jul 31 13:34:27 +0000 2018,1.024290e+18,1.024290e+18,RT @mashable: Someone from 'The Office' actual...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,10,...,Mike Santos,mikesantosx71,2419,2428,4,Thu May 25 14:37:29 +0000 2017,5993,False,2153,0
4,Tue Jul 31 13:34:28 +0000 2018,1.024290e+18,1.024290e+18,RT @_missj0hnson: Iâ€™m at Starbucks asking fo...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,25306,...,Soots,DaAverageDingus,314,722,7,Tue Mar 15 01:14:02 +0000 2011,6285,False,33503,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11094,Tue Jul 31 13:34:36 +0000 2018,1.024290e+18,1.024290e+18,RT @AnnieOakley2015: @AVoiceForAllGA Following...,False,"{'hashtags': [{'text': 'GA10', 'indices': [93,...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,5,...,TrumpNeck,Francis_3,802,1353,7,Thu May 28 12:02:54 +0000 2009,79149,False,6491,2
11095,Tue Jul 31 13:34:15 +0000 2018,1.024290e+18,1.024290e+18,RT @BillOReilly: What exactly is Donald Trump ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",False,8393,...,atm,allen_mcdowell,429,599,12,Wed Oct 10 11:37:34 +0000 2012,2029,False,5065,0
11096,Tue Jul 31 13:34:14 +0000 2018,1.024290e+18,1.024290e+18,"Paradoxically, your good intentions may stand ...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.twittascope.com"" rel=""nofo...",False,0,...,Jan15î„… .,yellow_naeeeee,135,94,0,Mon Sep 23 22:37:49 +0000 2013,90,False,4265,0
11097,Tue Jul 31 13:34:35 +0000 2018,1.024290e+18,1.024290e+18,RT @enblocibc: Less than 4 days to go for the ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,18,...,âœ¨MC Capital Venturesâœ¨,MCCapitalV,505,243,8,Thu Feb 01 23:58:16 +0000 2018,15829,False,13604,0


In [9]:
df['metadata'] = df['metadata'].apply(lambda x: eval(x))
metadata = df['metadata']
print(metadata[0].keys())
print(metadata[0].values())
# iso_language_code is same as lang column

dict_keys(['iso_language_code', 'result_type'])
dict_values(['en', 'recent'])


In [10]:
d_lang = dict()
for i in df['lang']:
    if i not in d_lang.keys():
        d_lang[i] = 1
    else:
        d_lang[i] += 1
        
# print(d_lang)
lang_mod = []
for i in df['lang']:
    lang_mod.append(d_lang[i])

df['lang_mod'] = lang_mod
df

,created_at,id,id_str,text,truncated,entities,metadata,source,is_quote_status,retweet_count,...,user_screen_name,user_followers_count,user_friends_count,user_listed_count,user_created_at,user_favourites_count,user_verified,user_statuses_count,hashtags,lang_mod
0,Tue Jul 31 13:34:34 +0000 2018,1.024290e+18,1.024290e+18,RT @EdwardTHardy: The 7th US Circuit Court of ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,113,...,queenfancygirl,153,264,7,Thu Mar 18 19:16:31 +0000 2010,32984,False,31308,0,10623
1,Tue Jul 31 13:34:14 +0000 2018,1.024290e+18,1.024290e+18,RT @VenomMovie: The world has enough superhero...,False,"{'hashtags': [{'text': 'Venom', 'indices': [64...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,5902,...,ikaykhairil,780,382,12,Wed Mar 17 03:27:51 +0000 2010,6648,False,87272,1,10623
2,Tue Jul 31 13:34:40 +0000 2018,1.024290e+18,1.024290e+18,RT @FutbolBible: Teachers vs Students match &a...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",False,3745,...,ch100897,255,246,1,Sun Mar 03 09:23:03 +0000 2013,5426,False,1731,0,10623
3,Tue Jul 31 13:34:27 +0000 2018,1.024290e+18,1.024290e+18,RT @mashable: Someone from 'The Office' actual...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com"" rel=""nofollow"">Tw...",False,10,...,mikesantosx71,2419,2428,4,Thu May 25 14:37:29 +0000 2017,5993,False,2153,0,10623
4,Tue Jul 31 13:34:28 +0000 2018,1.024290e+18,1.024290e+18,RT @_missj0hnson: Iâ€™m at Starbucks asking fo...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,25306,...,DaAverageDingus,314,722,7,Tue Mar 15 01:14:02 +0000 2011,6285,False,33503,0,10623
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11094,Tue Jul 31 13:34:36 +0000 2018,1.024290e+18,1.024290e+18,RT @AnnieOakley2015: @AVoiceForAllGA Following...,False,"{'hashtags': [{'text': 'GA10', 'indices': [93,...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,5,...,Francis_3,802,1353,7,Thu May 28 12:02:54 +0000 2009,79149,False,6491,2,10623
11095,Tue Jul 31 13:34:15 +0000 2018,1.024290e+18,1.024290e+18,RT @BillOReilly: What exactly is Donald Trump ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/iphone"" r...",False,8393,...,allen_mcdowell,429,599,12,Wed Oct 10 11:37:34 +0000 2012,2029,False,5065,0,10623
11096,Tue Jul 31 13:34:14 +0000 2018,1.024290e+18,1.024290e+18,"Paradoxically, your good intentions may stand ...",False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://www.twittascope.com"" rel=""nofo...",False,0,...,yellow_naeeeee,135,94,0,Mon Sep 23 22:37:49 +0000 2013,90,False,4265,0,10623
11097,Tue Jul 31 13:34:35 +0000 2018,1.024290e+18,1.024290e+18,RT @enblocibc: Less than 4 days to go for the ...,False,"{'hashtags': [], 'symbols': [], 'user_mentions...","{'iso_language_code': 'en', 'result_type': 're...","<a href=""http://twitter.com/download/android"" ...",False,18,...,MCCapitalV,505,243,8,Thu Feb 01 23:58:16 +0000 2018,15829,False,13604,0,10623


In [11]:
df.columns

Index(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities',
       'metadata', 'source', 'is_quote_status', 'retweet_count',
       'favorite_count', 'lang', 'user_name', 'user_screen_name',
       'user_followers_count', 'user_friends_count', 'user_listed_count',
       'user_created_at', 'user_favourites_count', 'user_verified',
       'user_statuses_count', 'hashtags', 'lang_mod'],
      dtype='object')

## Part-1
*Vectorize tweets using only meta data*

In [12]:
def get_features(df, df_truth):
    """
    Function to return a matrix of dimensions (number of tweets, number of chosen features)
    Input parameters to this funcion are to be chosen as per requirement (Example: Loaded dataframe of the dataset) 
    """

    # your code here
    new_df = df.drop(['created_at', 'id', 'id_str', 'text', 'truncated', 'entities', 'metadata', 'source', 'lang', 'user_name', 'user_screen_name', 'user_created_at'], axis=1)
    print(new_df.columns)
    x = new_df.to_numpy()
    y = df_truth.to_numpy()
    return x,y
    
    
X,y = get_features(df, df_truth)
print(X[0])
print(y[1])

Index(['is_quote_status', 'retweet_count', 'favorite_count',
       'user_followers_count', 'user_friends_count', 'user_listed_count',
       'user_favourites_count', 'user_verified', 'user_statuses_count',
       'hashtags', 'lang_mod'],
      dtype='object')
[False 113 0 153 264 7 32984 False 31308 0 10623]
[0]


In [13]:
print(len(X), len(y))

11099 11099


Perform KNN using the vector obtained from get_features() function. Following are the steps to be followed:
1. Normalise the vectors
2. Split the data into training and test to estimate the performance.
3. Fit the Nearest Neughbiurs module to the training data and obtain the predicted class by getting the nearest neighbours on the test data.
4. Report the accuracy, chosen k-value and method used to obtain the predicted class. Hint: Plot accuracies for a range of k-values. 

In [14]:
# your code here
X = normalize(X)
X

array([[0.00000000e+00, 2.41958973e-03, 0.00000000e+00, ...,
        6.70376242e-01, 0.00000000e+00, 2.27462847e-01],
       [0.00000000e+00, 6.67883241e-02, 0.00000000e+00, ...,
        9.87589058e-01, 1.13162189e-05, 1.20212194e-01],
       [0.00000000e+00, 2.96590954e-01, 0.00000000e+00, ...,
        1.37089170e-01, 0.00000000e+00, 8.41304594e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.72530266e-01, 0.00000000e+00, 9.27875502e-01],
       [0.00000000e+00, 7.68369196e-04, 0.00000000e+00, ...,
        5.80716363e-01, 0.00000000e+00, 4.53465887e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        7.95474413e-01, 0.00000000e+00, 6.04458132e-01]])

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)
print(len(X_train), len(X_test))

8879 2220


In [16]:
class KNN():
    def __init__(self, k=3, p=2):
        self.k = k
        self.p = p
        
    def distance(self, point):
        # return top k least distances of given point from all the points
        arr = list()
        for i in range(len(self.X)):
            dist = sum(abs(self.X[i] - point)**self.p)
            dist = dist**(1/self.p)
            arr.append([i,dist])
          
        arr = sorted(arr, key = lambda x: x[1])
        return arr[0:self.k]
    
    def classifier(self, arr):
        class_arr = [y_train[i[0]][0] for i in arr]
        return Counter(class_arr).most_common(1)[0][0]
    
    def fit(self, X, y):
        self.X = X
        self.y = y
        
    def predict(self, X):
        y_pred = np.empty(X.shape[0])
        
        for ind, i in enumerate(X):
            arr = self.distance(i)
            y_pred[ind] = self.classifier(arr)
            if(ind % 100 == 0):
                print(ind)
        return y_pred            

In [17]:
knn = KNN()
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
acc = accuracy_score(y_test, y_pred)
acc

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


0.9536036036036036

### experimenting for different values of k

In [ ]:
k = np.arange(1,50)
accuracy = list()
for i in k:
    knn = KNN(k = i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracy.append(acc)
    
plt.plot(k, accuracy)
plt.xLabel('K')
plt.yLabel('Accuracy')
plt.show()

KeyboardInterrupt: 

### experimenting for different values of p

In [ ]:
p = np.arange(1,10)
accuracy = list()
for i in p:
    knn = KNN()
    knn.fit(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracy.appendnd(acc)
    
plt.plot(k, accuracy)
plt.xLabel('K')
plt.yLabel('Accuracy')
plt.show()

## Part-2
Vectorize tweets based on the text. More details and reference links can be checked on the Tasks list in the start of the notebook

In [19]:
!pip install PyStemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.0/303.0 KB 10.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for PyStemmer: filename=PyStemmer-2.2.0.1-cp38-cp38-linux_x86_64.whl size=598200 sha256=1d106e6d77547a27294e7e7acdf30d5bb9dec1d3ed996e69c4851d81b3f139ad
  Stored in directory: /root/.cache/pip/wheels/78/04/32/a81f10f01775fcadba622dbcf8305f8053ab1db21b20a25fc4
Successfully built PyStemmer


In [20]:
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import *
from Stemmer import Stemmer
import re

try:
    nltk.download('stopwords')
except:
    pass
stop_words = set(stopwords.words('english'))
stop_words.add('rt')
stemmer = Stemmer('porter')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [21]:
def tokenise(text):
    text = text.lower()
    text = re.sub(r'&nbsp;|&lt;|&gt;|&amp;|&quot;|&apos;', r' ', text) # removing html entities
    text = re.sub(r'http[s]?\S*[\s | \n]', r' ', text) # removing links
    text = re.sub("(#[A-Za-z0-9]+)|(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)","",text) # removing tags and hashtags
    text = re.split(r'[^A-Za-z0-9]+', text)
    final = ''
    for i in text:
        word = stemmer.stemWord(i)
        if word not in stop_words:
            word += ' '
            final += word
    return final

In [23]:
from math import log2
from collections import defaultdict
from sklearn.decomposition import PCA

def BoW(tweets):
    d = dict()
    l = len(tweets)
    for ind, words in enumerate(tweets):
        words = re.split(r'[^A-Za-z0-9]+', words)
        for i in words:
            if i not in d.keys():
                d[i] = set()
            d[i].add(ind)
    for i in d.keys():
        d[i] = log2(l / len(d[i]))
    word_set = list(d.keys())
    return d, word_set

def tdf(tweets, word_set):
    x = list()
    d = dict()
    for i in word_set:
        d[i] = 0
    for ind, words in enumerate(tweets):
        words = re.split(r'[^A-Za-z0-9]+', words)
        for i in words:
            d[i] += 1
        d = {k: v / total for total in (sum(d.values(), 0.0),) for k,v in d.items()}
        x.append(list(d.values()))
        d = d.fromkeys(d, 0)
    return x

def tf_idf(word_docs, tdf_arr, word_set):
    for ind, doc in enumerate(tdf_arr):
        for ind2, word in enumerate(doc):
            if word != 0:
                idf = word_docs[word_set[ind2]]
                tdf_arr[ind][ind2] *= idf
        if(ind %100 == 0):
          print(ind)
            
    return tdf_arr

In [24]:
def tweet_vectoriser(tweets):
    """
    Funtion to return a matrix of dimensions (number of tweets, number of features extracted per tweet)
    Following are the steps for be followed:
    1. Remove links, tags and hashtags from each tweet.
    2. Apply TF-IDF on the tweets to extract a vector. 
    3. Perform dimensionality reduction on the obtained vector. 
    Input parameters to this funcion are to be chosen as per requirement (Example: Array of tweets) 
    """
    # your code here
    for ind, i in enumerate(tweets): # tokenise
        i = tokenise(i)
        tweets[ind] = i
        
    word_docs, word_set = BoW(tweets)
    
    tdf_arr = tdf(tweets, word_set)
    tdf_arr = tf_idf(word_docs, tdf_arr, word_set)
#     final_vector = list()
#     for ind, i in enumerate(tdf_arr):
#         final_vector.append(list(i.values()))
        
#     return np.array(tdf_arr)
    return tdf_arr
        
    

tweets = df['text'].to_list()

# print(tweets[:10], end = '\n\n')
tdf_arr = tweet_vectoriser(tweets)
# final_vector = tweet_vectoriser(tweets)
# final_vector[:5]

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000


In [32]:
print(len(tdf_arr))

11099


Perform KNN using the vector obtained from tweet_vectoriser() function. Following are the steps to be followed:

1. Normalise the vectors
2. Split the data into training and test to estimate the performance.
3. Fit the Nearest Neughbiurs module to the training data and obtain the predicted class by getting the nearest neighbours on the test data.
4. Report the accuracy, chosen k-value and method used to obtain the predicted class. Hint: Plot accuracies for a range of k-values.

In [42]:
# your code here
from sklearn.decomposition import PCA
tdf_arr = np.array(tdf_arr)
y = df_truth.to_numpy()
tdf_arr = normalize(tdf_arr)

# dimensionality reduction
pca = PCA(n_components=20)
pca.fit(tdf_arr)
tdf_arr = pca.transform(tdf_arr)
print(tdf_arr.shape)

(11099, 20)


In [44]:
X_train, X_test, y_train, y_test = train_test_split(tdf_arr, y, test_size=0.2, random_state=37)
print(len(X_train), len(X_test))

8879 2220


In [45]:
knn = KNN()
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
acc = accuracy_score(y_test, y_pred)
acc

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


0.9274774774774774

### experimenting for different values of k

In [ ]:
k = np.arange(1,50)
accuracy = list()
for i in k:
    knn = KNN(k = i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracy.append(acc)
    
plt.plot(k, accuracy)
plt.xLabel('K')
plt.yLabel('Accuracy')
plt.show()

## Part-3
### Subpart-1

Combine both the vectors obtained from the tweet_vectoriser() and get_features()

In [46]:
# your code here
X

array([[0.00000000e+00, 2.41958973e-03, 0.00000000e+00, ...,
        6.70376242e-01, 0.00000000e+00, 2.27462847e-01],
       [0.00000000e+00, 6.67883241e-02, 0.00000000e+00, ...,
        9.87589058e-01, 1.13162189e-05, 1.20212194e-01],
       [0.00000000e+00, 2.96590954e-01, 0.00000000e+00, ...,
        1.37089170e-01, 0.00000000e+00, 8.41304594e-01],
       ...,
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        3.72530266e-01, 0.00000000e+00, 9.27875502e-01],
       [0.00000000e+00, 7.68369196e-04, 0.00000000e+00, ...,
        5.80716363e-01, 0.00000000e+00, 4.53465887e-01],
       [0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
        7.95474413e-01, 0.00000000e+00, 6.04458132e-01]])

In [47]:
tdf_arr

array([[-1.27599628e-02, -8.63947526e-03, -2.48583929e-03, ...,
         8.70226436e-03,  9.06673783e-03, -1.66624808e-02],
       [ 9.77240041e-01,  5.24844940e-02,  2.42160954e-02, ...,
        -5.45255347e-03,  9.26525141e-03,  1.06613763e-02],
       [-1.08071289e-02, -1.09604880e-02, -6.02587775e-03, ...,
         2.03152671e-02,  3.11450184e-02, -2.24298048e-02],
       ...,
       [-7.16953725e-02,  9.84768094e-01,  3.25883378e-02, ...,
         2.48855143e-04,  2.78693716e-03, -5.18309967e-03],
       [ 4.62548973e-02, -7.18861899e-03, -5.62064280e-03, ...,
        -5.92773790e-02, -1.11808856e-02,  7.34570437e-02],
       [-2.58806169e-02, -2.54346885e-02, -5.52974775e-03, ...,
        -6.58730427e-03, -7.84945492e-03,  9.62314782e-03]])

In [50]:
X = np.concatenate((X, tdf_arr), axis=1)

In [53]:
print(X.shape)

(11099, 31)


Perform KNN using the vector obtained in the previous step. Following are the steps to be followed:

1. Normalise the vectors
2. Split the data into training and test to estimate the performance.
3. Fit the Nearest Neughbiurs module to the training data and obtain the predicted class by getting the nearest neighbours on the test data.
4. Report the accuracy, chosen k-value and method used to obtain the predicted class. Hint: Plot accuracies for a range of k-values.

In [56]:
# your code here
X = normalize(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=37)
print(len(X_train), len(X_test))

8879 2220


In [57]:
knn = KNN()
knn.fit(X_train, y_train)

y_pred = knn.predict(X_test)
acc = accuracy_score(y_test, y_pred)
acc

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200


0.9509009009009008

### experimenting with values of k

In [ ]:
k = np.arange(1,50)
accuracy = list()
for i in k:
    knn = KNN(k = i)
    knn.fit(X_train, y_train)
    y_pred = knn.predict(X_test)
    acc = accuracy_score(y_test, y_pred)
    accuracy.append(acc)
    
plt.plot(k, accuracy)
plt.xLabel('K')
plt.yLabel('Accuracy')
plt.show()

### Subpart-2

Explain the differences between the accuracies obtained in each part above based on the features used.